In [113]:
import pandas as pd

#dest = pd.read_csv('destinations.csv', delimiter=',')
'''test = []

for chunk in  pd.read_csv('test.csv', delimiter=',', chunksize=10**6):
        test.append(chunk)
test = pd.concat(test)
'''

"test = []\n\nfor chunk in  pd.read_csv('test.csv', delimiter=',', chunksize=10**6):\n        test.append(chunk)\ntest = pd.concat(test)\n"

In [115]:
chunksize = 10**5

skiprows = [i for i in range(10**5) if i%10==1]
train = pd.read_csv('train.csv', delimiter=',', skiprows=skiprows, chunksize=chunksize)


features = None
target = None

for t in train:
        print(t.head())
        target = t['hotel_cluster']
        features = t.drop(['hotel_cluster'], axis=1)
        break


             date_time  site_name  posa_continent  user_location_country  \
0  2014-08-11 08:22:12          2               3                     66   
1  2014-08-11 08:24:33          2               3                     66   
2  2014-08-09 18:05:16          2               3                     66   
3  2014-08-09 18:08:18          2               3                     66   
4  2014-08-09 18:13:12          2               3                     66   

   user_location_region  user_location_city  orig_destination_distance  \
0                   348               48862                  2234.2641   
1                   348               48862                  2234.2641   
2                   442               35390                   913.1932   
3                   442               35390                   913.6259   
4                   442               35390                   911.5142   

   user_id  is_mobile  is_package      ...        srch_children_cnt  \
0       12          0      

In [116]:
#TODO: Handle dt errors
#Convert to date_time
features['date_time'] = pd.to_datetime(features['date_time'], errors='coerce')
features['srch_ci'] = pd.to_datetime(features['srch_ci'], errors='coerce')
features['srch_co'] = pd.to_datetime(features['srch_co'], errors='coerce')

#Add new features
features['length_of_stay'] = features['srch_co']-features['srch_ci']
features['length_of_stay'] = features['length_of_stay'].dt.days
features['season'] = features['srch_ci'].dt.month
features['year'] = features['date_time'].dt.year
features['month'] = features['date_time'].dt.month
features['day'] = features['date_time'].dt.day
features['hour'] = features['date_time'].dt.hour
features['is_couple'] = features['srch_adults_cnt'].map(lambda x: x==2) & features['srch_children_cnt'].map(lambda x: x==0) & features['srch_rm_cnt'].map(lambda x: x==1)


#Drop old features and remove NaN
features.drop(['date_time', 'srch_ci', 'srch_co'], axis=1, inplace=True)
features.fillna(-1, inplace=True)



In [117]:
from sklearn.feature_selection import SelectKBest
print(features.head())

sel = SelectKBest(k=25)
sel.fit(features, target)
reduced_features = sel.transform(features)

print(reduced_features)
print(sel.scores_)

   site_name  posa_continent  user_location_country  user_location_region  \
0          2               3                     66                   348   
1          2               3                     66                   348   
2          2               3                     66                   442   
3          2               3                     66                   442   
4          2               3                     66                   442   

   user_location_city  orig_destination_distance  user_id  is_mobile  \
0               48862                  2234.2641       12          0   
1               48862                  2234.2641       12          0   
2               35390                   913.1932       93          0   
3               35390                   913.6259       93          0   
4               35390                   911.5142       93          0   

   is_package  channel    ...      hotel_continent  hotel_country  \
0           1        9    ...      

In [118]:
from sklearn.cross_validation import train_test_split

train_X, test_X, train_y, test_y = train_test_split(features, target, test_size=.25)
print(train_X.shape)


(75000, 27)


In [119]:
"""from sklearn.svm import SVC
svc = SVC(C=0.1)
svc.fit(train_X, train_y)
print(svc.score(test_X, test_y))"""

'from sklearn.svm import SVC\nsvc = SVC(C=0.1)\nsvc.fit(train_X, train_y)\nprint(svc.score(test_X, test_y))'

In [126]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(train_X, train_y)
print(clf.score(test_X, test_y))

0.0596


In [125]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(max_depth=15)
rfc.fit(train_X, train_y)
print(rfc.score(test_X, test_y))

0.22376
